In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os
from tqdm import tqdm_notebook as tqdm
#from codecs import open
from lxml import etree

from nlppln.utils import out_file_name, create_dirs

In [ ]:
with open('/home/jvdzwaan/data/dbnl_ocr/raw/ocr-with-title-page/_aio001jver01_01.txt') as f:
    ls = f.readlines()

with open('/home/jvdzwaan/data/dbnl_ocr/raw/notes/_aio001jver01_01.txt') as f:
    notes = f.readlines()

print(len(ls))
print(len(notes))

In [ ]:
with open('/home/jvdzwaan/data/dbnl_ocr/raw/ocr-with-title-page/rade001gera01_01.txt') as f:
    ls = f.readlines()

with open('/home/jvdzwaan/data/dbnl_ocr/raw/notes/rade001gera01_01.txt') as f:
    notes = f.readlines()

print(len(ls))
print(len(notes))

In [ ]:
# remove empty lines from lines
lines = []

for line in ls:
    if line.strip() != '':
        lines.append(line)
print(len(lines))

In [ ]:
avg_line_length = np.mean([len(line) for line in lines])
print(avg_line_length)

In [ ]:
from ochre.dbnl import Note, to_fragment, complete, get_repeated, extend_lines

repeated = get_repeated(notes)

In [ ]:
%%time
# Calculate edit distance between notes and lines
import edlib

# get initial edit distances per line
line_data = pd.DataFrame()
for i, n in enumerate(tqdm(notes)):
    line_data[i] = [edlib.align(n, line)['editDistance'] for line in lines]

In [ ]:
line_data.head()

In [ ]:
# initialize note objects
scores = np.full_like(np.empty((len(notes), len(lines))), -1, dtype=np.int)
print(scores.shape)
prev_note = None
ns = []
text = ''.join(lines)
for i, n in enumerate(notes):
    note = Note(n, i, len(lines))
    if n in repeated:
        note.repeated = True
    else:
        note.repeated = False
        
    note.previous = prev_note
    prev_note = note
    note.candidates = []
    
    ns.append(note)

ns.reverse()
for i, n in enumerate(ns):
    if i > 0:
        n.next = ns[i-1]
ns.reverse()

In [ ]:
print(scores[0,0])

In [ ]:
%%time
import re
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
pr = PartialRatio()

num = 0

for i, n in enumerate(tqdm(ns)):
    #print(n)
    if not n.repeated:
        for j, idx in enumerate(line_data[i].sort_values().index[:10]):
            scores[i, idx] = pr.get_raw_score(lines[idx], n.text)
            #print(idx, lines[idx].strip(), score)
            
            # If the line starts with a number, sometimes the score is calculated incorrectly
            if scores[i, idx] < 90 and lines[idx][0].isdigit():
                sc = pr.get_raw_score(re.sub(r'^\d+', '', lines[idx]), n.text)
                if sc > 90:
                    #print('Fixed score', lines[idx])
                    scores[i, idx] = sc
            if scores[i, idx] > 90:
                n.candidates.append(idx)
                num += 1
                #print('Added index', j)
                #if j > 1:
                #    print(n.text)
                #    print('j', j)
                #    print(idx, lines[idx].strip(), score)
                #    print('---')
                #break
        n.candidates.sort()
print(num)

In [ ]:
c = 0
for n in tqdm(ns):
    if n.candidates != []:
        done = complete(n.text, to_fragment(lines, n.candidates))
        if done:
            n.lines = n.candidates
            n.complete(lines)
            c += 1
        #print(n)
        #print(n.text)
        #print(n.get_search_indices())
        #print(n.lines)
        #print(n.candidates)
        #print(to_fragment(lines, n.candidates))
        #print(complete(n.text, to_fragment(lines, n.candidates)))
        #if (complete(n.text, to_fragment(lines, n.candidates))):
        #    c += 1
        #print(n.next)
        #print('---')
print(c)

In [ ]:
c = 0
for n in tqdm(ns):
    if not n.done:
        c += 1
        print(n)
        print(n.text)
        print(n.get_search_indices())
        print(n.lines)
        #print(n.candidates)
        print(n.to_fragment(lines))
        print('---')
print(c)

In [ ]:
scores[16, 1078]

In [ ]:
lines[1078]

In [ ]:
c = 0
nf = 0
s = 0
other = []
for n in ns:
    if n.done:
        c += 1
    if n.lines == []:
        nf += 1
    if n.short:
        s += 1
    if not n.done and n.lines != []:
        other.append(n.index)
print('Complete', c)
print('Not found', nf)
print('Short', s)
print('Incomplete', len(other))
print('Total', np.sum([c, nf, len(other)]))

In [ ]:
for n in ns:
    if n.lines == []:
        print(n.index)
        print(n.text)
        print(n.to_fragment(lines))
        print(n.lines)
        print('---')

In [ ]:
for i in other:
    n = ns[i]
    print(n.index)
    print(n.text)
    print(n.to_fragment(lines))
    print(n.added)
    print(n.lines)
    print('---')

In [ ]:
import re
pr.get_raw_score(re.sub(r'^\d+', '', lines[6476]), ns[325].text)

In [ ]:
lines[14190]

In [ ]:
print(ns[589].text)
print(ns[589].to_fragment(lines))
print(ns[589].lines)
print(ns[589].done)

In [ ]:
a = lines[1078].replace('`', '')
a

In [ ]:
b = ns[16].text.replace('‘', '')
b

In [ ]:
repr(a)

In [ ]:
repr(b)

In [ ]:
edlib.align(a, b, task='path')

In [ ]:
pr.get_raw_score(ns[638].text, lines[1490])

In [ ]:
pr.get_raw_score(a[3:], b)

In [ ]:
import re

print('Matching best first')
f = 0
l = 0
for i, n in enumerate(tqdm(ns)):
    #print(n.index)
    #print(n.lines_to_check[:10])
    #print('no of lines to check', len(n.lines_to_check))
    if not n.done:
        #print(n.get_search_indices())
        for idx in n.get_search_indices():
            if idx not in n.lines:
                if scores[i, idx] == -1:
                    scores[i, idx] = pr.get_raw_score(lines[idx], n.text)
                    
                    if scores[i, idx] < 90 and lines[idx][0].isdigit():
                        sc = pr.get_raw_score(re.sub(r'^\d+', '', lines[idx]), n.text)
                        if sc > 90:
                            #print('Fixed score', lines[idx])
                            scores[i, idx] = sc
    
                if scores[i, idx] > 90:
                    l += 1
                    n.lines.append(idx)
                    n.complete(lines)
                    break
print('Added {} lines.'.format(l))
print('Found {} "complete" notes.'.format(f))

In [ ]:
print('Matching lines incomplete notes/notes without candidates')
f = 0
l = 0
for i, n in enumerate(tqdm(ns)):
    #print(n.index)
    #print(n.lines_to_check[:10])
    #print('no of lines to check', len(n.lines_to_check))
    if not n.done:
        #print(n.get_search_indices())
        for idx in n.get_search_indices():
            if idx not in n.lines:
                if scores[i, idx] == -1:
                    scores[i, idx] = pr.get_raw_score(lines[idx], n.text)
                        
                if scores[i, idx] > 90:
                    l += 1
                    n.lines.append(idx)
                    if n.complete(lines):
                        f += 1
                        #print('Complete', n.index)
                        #print(n.lines)
                        #print(n.to_fragment(lines))
                        #print('Checked:', checked)
                        break
                    
                    if len(n.text) < avg_line_length:
                        n.short = True
                        print('short', n)
                        break
        #if n.index == 434:
        #    print(n.lines)
        #print(n.lines)
        #print('Checked:', checked)
print('Added {} lines.'.format(l))
print('Found {} "complete" notes.'.format(f))

In [ ]:
import copy

print('Extending lines')
added = 0

for n in tqdm(ns):
    #print(n)
    #print(n.lines)
    if n.lines != []:
        ls = copy.copy(n.lines)
        extended = extend_lines(n.lines, len(lines), 25)
        
        for idx in extended:
            #print(idx)
            if idx not in n.lines:
                #print('Checking')
                ed = n.ed(lines)
                #print('ed', ed)
                ls.append(idx)
                ls.sort()
                #print(ls)
                new_ed = edlib.align(to_fragment(lines, ls), n.text)['editDistance']
                #print('new_ed', new_ed)
                if new_ed < ed:
                    #print('Adding', idx)
                    added += 1
                    n.lines.append(idx)
                    n.lines.sort()
                    n.complete(lines)
                else:
                    ls.remove(idx)
        #print(n.lines)
print('Added {} lines.'.format(added))

In [ ]:
import copy

print('Extending lines')
added = 0

for n in tqdm(ns):
    if n.lines != []:
        #print(n.text)
        #print(lines[n.lines[0]])
        #print(scores[n.index, n.lines[0]])
        #print(n.lines)
        ls = copy.copy(n.lines)
        extended = list(n.get_search_indices())
        before = [idx for idx in extended if idx < n.lines[0]]
        before.reverse()
        inside = [idx for idx in extended if idx > n.lines[0] and idx < n.lines[-1]]
        after = [idx for idx in extended if idx > n.lines[-1]]
        #print(n.lines)
        #print(before)
        #print(inside)
        #print(after)
        for idx in inside + before + after:
            #print(idx)
            if idx not in n.lines:
                #print('Checking')
                ed = n.ed(lines)
                #print('ed', ed)
                ls.append(idx)
                ls.sort()
                #print(ls)
                new_ed = edlib.align(to_fragment(lines, ls), n.text)['editDistance']
                #print('new_ed', new_ed)
                if new_ed < ed:
                    #print('Adding', idx)
                    added += 1
                    n.lines.append(idx)
                    n.lines.sort()
                    n.added.append(idx)
                    n.complete(lines)
                else:
                    ls.remove(idx)
        #if n.added != []:
        #    print(n)
        #    print(n.added)
        #    print(n.lines)
        #    print('---')
print('Added {} lines.'.format(added))

In [ ]:
for n in ns:
    if n.lines == []:
        print(n)
        print(n.added)
        print(n.lines)
        print(n.text)
        print(n.to_fragment(lines))
        print(n.complete(lines))
        print('---')

In [ ]:
for n in ns:
    print(n)
    print(n.added)
    print(n.lines)
    print(n.text)
    print(n.to_fragment(lines))
    print(n.complete(lines))
    print('---')

In [ ]:
n = ns[434]
print(n.candidates)
print(n.lines)
print(n.get_search_indices())
print(to_fragment(lines, n.lines))
#print(to_fragment(lines, n.get_search_indices()))

In [ ]:
#ls = [9265, 9266, 9267, 9268, 9269, 9270, 9271, 9272, 9273, 9274, 9275, 9276, 9277, 3919]
ls = [9265, 9266, 9267, 9268, 9272, 9273, 9275, 9277, 9319]
print(to_fragment(lines, ls))
print('---')
print(ns[434].text)
complete(ns[434].text, to_fragment(lines, ls))

In [ ]:
lines[977]

In [ ]:
%%time
# match entire notes: quick way to distribute the notes over the text
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
pr = PartialRatio()

prev_note = None
ns = []
text = ''.join(lines)
for i, n in enumerate(notes):
    note = Note(n, i, len(lines))
    score = pr.get_raw_score(note.text, text)
    print(i, 'score', score)
    if score > 90:
        note.found = True
    else:
        note.found = False
    if n in repeated:
        note.repeated = True
    else:
        note.repeated = False
        
    note.previous = prev_note
    prev_note = note
    
    ns.append(note)

ns.reverse()
for i, n in enumerate(ns):
    if i > 0:
        n.next = ns[i-1]
ns.reverse()

In [ ]:
for n in ns:
    print(n, n.found, n.repeated)

In [ ]:
# roughly match found, unrepeated notes

from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
pr = PartialRatio()

for n in tqdm(ns):
    print(n)
    #print(n.lines_to_check[:10])
    #print('no of lines to check', len(n.lines_to_check))
    if n.found and not n.repeated:
        checked = 0
        for idx in range(len(lines)):
            if idx > n.get_search_start():
                checked += 1
                score = pr.get_raw_score(lines[idx], n.text)
                if score > 90:
                    n.lines.append(idx)
                    if n.complete(lines):
                        #print('Complete')
                        #print(n.lines)
                        #print(n.to_fragment(lines))
                        #print('Checked:', checked)
                        break
        print(n.lines)
        print('Checked:', checked)

In [ ]:
list(ns[10].get_search_indices())

In [ ]:
# roughly match unprepeated notes that didn't have an initial match

for n in tqdm(ns):
    print(n)
    #print(n.lines_to_check[:10])
    #print('no of lines to check', len(n.lines_to_check))
    if not n.found and not n.repeated:
        checked = 0
        for idx in n.get_search_indices():
            checked += 1
            score = pr.get_raw_score(lines[idx], n.text)
            if score > 90:
                n.lines.append(idx)
                n.found = True
                if n.complete(lines):
                    #print('Complete')
                    #print(n.lines)
                    #print(n.to_fragment(lines))
                    #print('Checked:', checked)
                    break
        print(n.lines)
        print('Checked:', checked)

In [ ]:
# match repeated notes

for n in tqdm(ns):
    print(n)
    #print(n.lines_to_check[:10])
    #print('no of lines to check', len(n.lines_to_check))
    if n.repeated:
        checked = 0
        for idx in n.get_search_indices():
            checked += 1
            score = pr.get_raw_score(lines[idx], n.text)
            if score > 90:
                n.lines.append(idx)
                n.found = True
                if n.complete(lines):
                    #print('Complete')
                    #print(n.lines)
                    #print(n.to_fragment(lines))
                    #print('Checked:', checked)
                    break
        print(n.lines)
        print('Checked:', checked)

In [ ]:
def extend_lines(indexes, length, num=3):
    #print('start')
    #print(indexes)
    #print('num:', num)
    result = []
    for i in indexes:
        #print('i', i)
        result.append(i)
        # lines before i
        start = max(0, i-num)      
        to_add = list(range(start, i))
        for n in to_add:
            result.append(n)
        #print(to_add)
        
        # lines after i
        end = min(length, i+num+1)
        #print('end', end)
        to_add = list(range(i, end))
        for n in to_add:
            result.append(n)
        #print(to_add)
        #print('---')
            
    result = list(set(result))
    result.sort()
            
    return(result)
        
extend_lines([5], 6)

In [ ]:
extend_lines([], 6)

In [ ]:
import copy

n = copy.deepcopy(ns[45])

In [ ]:
print(n.lines)

In [ ]:
extended = extend_lines(n.lines, len(lines), 5)
print(extended)
len(extended)

In [ ]:
print(n.text)

In [ ]:
for idx in n.lines:
    print(idx, lines[idx].strip())

In [ ]:
print(lines[1185])

In [ ]:
import copy

ls = copy.copy(n.lines)

for idx in extended:
    print(idx)
    if idx not in n.lines:
        print('Checking')
        ed = n.ed(lines)
        print('ed', ed)
        ls.append(idx)
        ls.sort()
        print(ls)
        new_ed = edlib.align(to_fragment(lines, ls), n.text)['editDistance']
        print('new_ed', new_ed)
        if new_ed <= ed:
            print('Adding', idx)
            n.lines.append(idx)
            n.lines.sort()
        else:
            ls.remove(idx)

In [ ]:
indices = [1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183]
fr = to_fragment(lines, indices)
fr2 = ns[45].text
print(edlib.align(fr.strip(), fr2.strip())['editDistance'])

In [ ]:
print(len(fr))

In [ ]:
indices = [1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182]
fr = to_fragment(lines, indices)
fr2 = ns[45].text
print(edlib.align(fr.strip(), fr2.strip())['editDistance'])

In [ ]:
print(len(fr))

In [ ]:
print(len(fr2))

In [ ]:
%%time
## find lines that were missed before
import copy

added = 0

for n in ns:
    print(n)
    print(n.lines)
    ls = copy.copy(n.lines)
    extended = extend_lines(n.lines, len(lines), 5)
    for idx in extended:
        #print(idx)
        if idx not in n.lines:
            #print('Checking')
            ed = n.ed(lines)
            #print('ed', ed)
            ls.append(idx)
            ls.sort()
            #print(ls)
            new_ed = edlib.align(to_fragment(lines, ls), n.text)['editDistance']
            #print('new_ed', new_ed)
            if new_ed <= ed:
                #print('Adding', idx)
                added += 1
                n.lines.append(idx)
                n.lines.sort()
            else:
                ls.remove(idx)
    print(n.lines)
print('Added', added)

## Putting it all together

In [ ]:
%%time
import edlib
import copy

import pandas as pd

from collections import Counter
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio

class Note():
    def __init__(self, text, index, source_text_length):
        self.index = index
        self.text = text
        self.source_text_length = source_text_length
        self.lines_to_check = []
        self.lines = []
        
        self.previous = None
        self.next = None
        
        self.done = False
        
    def __str__(self):
        return '(Note {}) {}'.format(self.index, self.text[:50].strip())
    
    def get_search_start(self):
        if self.previous is None:
            return 0
        elif self.previous.lines != []:
            return self.previous.lines[-1]
        else:
            return self.previous.get_search_start()
        
    def get_search_end(self):
        if self.next is None:
            return self.source_text_length
        elif self.next.lines != []:
            return self.next.lines[0]
        else:
            return self.next.get_search_end()
            
    def get_search_indices(self):
        return range((self.get_search_start()+ 1), self.get_search_end())
            
    def to_fragment(self, lines):
        return ''.join([lines[i] for i in self.lines])
    
    def ed(self, lines):
        fragment = self.to_fragment(lines)
        return edlib.align(self.text, fragment)['editDistance']
    
    def complete(self, lines, threshold=0.2):
        ed = self.ed(lines)
        score = ed/len(self.text)
        if score < threshold:
            self.done = True
        return self.done
    
def to_fragment(lines, indices):
    return ''.join([lines[i] for i in indices])

def complete(note, fragment, treshold=0.3):
    ed = edlib.align(note, fragment)['editDistance']
    #print(ed)
    #print(len(note))
    score = ed/len(note)
    #print(score)
    if score < treshold:
        return True
    return False

def get_repeated(notes):
    c = Counter()
    num_lines = 0

    for note in notes:
        c[note] += 1

    repeated = []
    ns = []
    for k, v in c.most_common():
        if v > 1:
            num_lines += v
            #print(repr(k), v)
            repeated.append(k)
        else:
            ns.append(k)
    #print('total number of lines:', num_lines)
    
    return repeated


def extend_lines(indexes, length, num=3):
    #print('start')
    #print(indexes)
    #print('num:', num)
    result = []
    for i in indexes:
        #print('i', i)
        result.append(i)
        # lines before i
        start = max(0, i-num)      
        to_add = list(range(start, i))
        for n in to_add:
            result.append(n)
        #print(to_add)
        
        # lines after i
        end = min(length, i+num+1)
        #print('end', end)
        to_add = list(range(i, end))
        for n in to_add:
            result.append(n)
        #print(to_add)
        #print('---')
            
    result = list(set(result))
    result.sort()
            
    return(result)


def remove_notes(ocr_file, notes_file, out_dir):
    with open(ocr_file) as f:
        ls = f.readlines()

    with open(notes_file) as f:
        notes = f.readlines()
        
    print('There are {} notes to be found.'.format(len(notes)))
        
    # remove empty lines
    lines = []

    for line in ls:
        if line.strip() != '':
            lines.append(line)
    print('The text contains {} lines.'.format(len(lines)))
    
    # get repeated notes
    repeated = get_repeated(notes)
    print('There are {} repeated notes.'.format(len(repeated)))
    
    pr = PartialRatio()
    
    print('Matching entire notes')

    f = 0
    prev_note = None
    ns = []
    text = ''.join(lines)
    for i, n in enumerate(tqdm(notes)):
        note = Note(n, i, len(lines))
        score = pr.get_raw_score(note.text, text)
        #print(i, 'score', score)
        if score > 90:
            note.found = True
            f += 1
        else:
            note.found = False
        if n in repeated:
            note.repeated = True
        else:
            note.repeated = False
        
        note.previous = prev_note
        prev_note = note
    
        ns.append(note)

    ns.reverse()
    for i, n in enumerate(ns):
        if i > 0:
            n.next = ns[i-1]
    ns.reverse()
    
    print('Found {} notes.'.format(f))
    
    # roughly match found, unrepeated notes
    print('Matching lines of found, unrepeated notes')
    pr = PartialRatio()

    f = 0
    l = 0
    for n in tqdm(ns):
        #print(n)
        #print(n.lines_to_check[:10])
        #print('no of lines to check', len(n.lines_to_check))
        if n.found and not n.repeated:
            checked = 0
            for idx in range(len(lines)):
                if idx > n.get_search_start():
                    checked += 1
                    score = pr.get_raw_score(lines[idx], n.text)
                    if score > 90:
                        l += 1
                        n.lines.append(idx)
                        if n.complete(lines):
                            f += 1
                            #print('Complete')
                            #print(n.lines)
                            #print(n.to_fragment(lines))
                            #print('Checked:', checked)
                            break
            #print(n.lines)
            #print('Checked:', checked)
    print('Added {} lines.'.format(l))
    print('Found {} "complete" notes.'.format(f))
            
    # roughly match unprepeated notes that didn't have an initial match
    f = 0
    l = 0
    print('Matching lines of unfound, unrepeated notes')
    for n in tqdm(ns):
        #print(n)
        #print(n.lines_to_check[:10])
        #print('no of lines to check', len(n.lines_to_check))
        if not n.found and not n.repeated:
            checked = 0
            for idx in n.get_search_indices():
                checked += 1
                score = pr.get_raw_score(lines[idx], n.text)
                if score > 90:
                    l += 1
                    n.lines.append(idx)
                    n.found = True
                    if n.complete(lines):
                        f += 1
                        #print('Complete')
                        #print(n.lines)
                        #print(n.to_fragment(lines))
                        #print('Checked:', checked)
                        break
            #print(n.lines)
            #print('Checked:', checked)
    
    print('Added {} lines.'.format(l))
    print('Found {} "complete" notes.'.format(f))
            
    # match repeated notes
    f = 0
    l = 0
    print('Matching repeated notes')
    for n in tqdm(ns):
        #print(n)
        #print(n.lines_to_check[:10])
        #print('no of lines to check', len(n.lines_to_check))
        if n.repeated:
            checked = 0
            for idx in n.get_search_indices():
                checked += 1
                score = pr.get_raw_score(lines[idx], n.text)
                if score > 90:
                    l += 1
                    n.lines.append(idx)
                    n.found = True
                    if n.complete(lines):
                        f += 1
                        #print('Complete')
                        #print(n.lines)
                        #print(n.to_fragment(lines))
                        #print('Checked:', checked)
                        break
            #print(n.lines)
            #print('Checked:', checked)
    
    print('Added {} lines.'.format(l))
    print('Found {} "complete" notes.'.format(f))
        
    print('Extending lines')
    added = 0

    for n in tqdm(ns):
        #print(n)
        #print(n.lines)
        ls = copy.copy(n.lines)
        extended = extend_lines(n.lines, len(lines), 5)
        for idx in extended:
            #print(idx)
            if idx not in n.lines:
                #print('Checking')
                ed = n.ed(lines)
                #print('ed', ed)
                ls.append(idx)
                ls.sort()
                #print(ls)
                new_ed = edlib.align(to_fragment(lines, ls), n.text)['editDistance']
                #print('new_ed', new_ed)
                if new_ed <= ed:
                    #print('Adding', idx)
                    added += 1
                    n.lines.append(idx)
                    n.lines.sort()
                else:
                    ls.remove(idx)
        #print(n.lines)
    print('Added {} lines.'.format(added))
    
    f = 0
    c = 0
    for n in ns:
        if n.found:
            f += 1
        if n.complete(lines):
            c += 1
    print('Found {} of {} notes (complete: {})'.format(f, len(notes), c))

        
    print('Removing notes')
    removed = []

    for n in ns:
        for idx in n.lines:
            removed.append(idx)
            
    # get the ocr text
    with open(ocr_file) as f:
        text = f.read()
        
    removed = list(set(removed))
    for idx in removed:
        l = lines[idx]
        text = text.replace(l, '')

    # save result
    create_dirs(out_dir)
    out = out_file_name(out_dir, ocr_file)
    #print(out)
    with open(out, 'w') as f:
        f.write(text)
        
    return removed

r = remove_notes('/home/jvdzwaan/data/dbnl_ocr/raw/ocr-with-title-page/_aio001jver01_01.txt', 
                 '/home/jvdzwaan/data/dbnl_ocr/raw/notes/_aio001jver01_01.txt',
                 '/home/jvdzwaan/data/dbnl_ocr/raw/ocr')
#print(r)

In [ ]:
import os

from tqdm import tqdm_notebook as tqdm

from nlppln.utils import get_files, out_file_name, create_dirs

in_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/ocr-without-title-page/'
notes_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/notes/'
out_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/ocr-1pass'

create_dirs(out_dir)

in_files = ['rade001gera01_01.txt', '_zev001198901_01.txt', '_tir001196201_01.txt',
            'looy001wond03_01.txt', 'potg001jczi10_01.txt', 'berg050jaro01_01.txt',
            '_tsj002195001_01.txt', '_jaa006199901_01.txt', '_taa006189101_01.txt',
            '_sep001197201_01.txt', 'oltm003memo05_01.txt', '_noo001189201_01.txt',
            'koni057heil01_01.txt', '_vla016197401_01.txt', '_bij005195501_01.txt']
in_files = [os.path.join(in_dir, f) for f in in_files]

with open('lines_removed_iteratively.txt', 'w') as f:
    for in_file in tqdm(in_files):
        out = out_file_name(out_dir, in_file)
    
        # is there a notes file?
        notes_file = os.path.join(notes_dir, os.path.basename(in_file))
        if os.path.isfile(notes_file):
            print('processing', in_file)
        
            removed = remove_notes(in_file, notes_file, out_dir)
            removed.sort()
            f.write(os.path.basename(out))
            f.write('\t')
            removed = [str(r) for r in removed]
            f.write(','.join(removed))
            f.write('\n')
        print('---')